### further 2022-2023 highlights

#### Advanced modelling

* extended parallelism, including parallel broadcasting to hierarchical data
* fully distributional probabilistic forecasts and metrics, skpro
* composable time series classifiers, regressors, distances, time series aligners
* benchmarking frameworks for comparing estimator performance

#### Marketplace and deployment features

* estimator search, estimator tags
* scikit-base interface for multiple libraries
* blueprint serialization and sharing
* fitted estimator serialization and sharing
* mlflow deployment via custom flavour

In [1]:
import warnings
warnings.filterwarnings("ignore")

---

# Advanced modelling features

## parallelism for multivariate and hierarchical broadcasting

univariate forecasters broadcast across variables if given multivariate data

example:

In [2]:
from sktime.datasets import load_longley
from sktime.forecasting.arima import ARIMA

_, y = load_longley()

y = y.drop(columns=["UNEMP", "ARMED", "POP"])

forecaster = ARIMA()
forecaster.fit(y, fh=[1, 2, 3])

# forecasters_ is a data frame with fitted ARIMA models
# entries are references to individual isntances of ARIMA, per variable
forecaster.forecasters_

,GNPDEFL,GNP
forecasters,ARIMA(),ARIMA()


by default, this is base python loop ... but we can use parallel backend!

In [3]:
forecaster = ARIMA()

# let's use joblib loky backend, with 2 workers
# parallelization configs are accessed via the scikit-base config interface

# backends are set via the backend:parallel config
forecaster.set_config(**{"backend:parallel": "loky"})  # or "multiprocessing", or "dask" (requires dask)
# backend params are set via the backend:parallel:params config
forecaster.set_config(**{"backend:parallel:params": {"n_jobs": 2}})  # passed to joblib.Parallel
# for documentation of the config interface, see set_config/get_config docstrings

ARIMA()

In [4]:
# fit/predict methods are now parallelized
forecaster.fit(y, fh=[1, 2, 3])
forecaster.forecasters_
# of course this is more useful for larger data

,GNPDEFL,GNP
forecasters,ARIMA(),ARIMA()


same for hierarchical data!

hierarchical = multiple time series by hierarchical scope or index, e.g., product line/category

(typical: 1.000s of low-level hierarchical categories)

![](./img/hierarchy.png)

In [5]:
from hierarchical_demo_utils import load_product_hierarchy
from sktime.forecasting.model_selection import temporal_train_test_split

y = load_product_hierarchy()

y_train, y_test = temporal_train_test_split(y, test_size=4)
y_train

Sales
Product line      Product group Date          
Food preparation  Hobs          2000-01  245.0
                                2000-02  144.0
                                2000-03  184.0
                                2000-04  265.0
                                2000-05  236.0
...                                        ...
Food preservation Fridges       2004-04  117.0
                                2004-05  126.0
                                2004-06  161.0
                                2004-07   94.0
                                2004-08  176.0

[224 rows x 1 columns]

sliced at a specific date:

In [6]:
# Multiindex slicing can become important when using hierarchical data!
y.loc[(slice(None), slice(None), "2000-01")]

Sales
Product line      Product group       
Food preparation  Hobs           245.0
                  Ovens          114.0
Food preservation Freezers       164.0
                  Fridges        136.0

Like for variables, `sktime` broadcasts simple models to hierarchical data:

In [7]:
from sktime.forecasting.ets import AutoETS

forecaster = AutoETS(auto=True)

forecaster.fit(y_train, fh=[1, 2, 3, 4])
y_pred = forecaster.predict()
y_pred

Sales
Product line      Product group Date               
Food preparation  Hobs          2004-09  121.444964
                                2004-10  120.502170
                                2004-11  119.559377
                                2004-12  118.616583
                  Ovens         2004-09  182.823610
                                2004-10  183.980932
                                2004-11  185.138253
                                2004-12  186.295575
Food preservation Freezers      2004-09  148.411369
                                2004-10  148.411369
                                2004-11  148.411369
                                2004-12  148.411369
                  Fridges       2004-09  139.052209
                                2004-10  139.052209
                                2004-11  139.052209
                                2004-12  139.052209

In [8]:
# forecasters_ has fitted ETS models
forecaster.forecasters_

forecasters
Food preparation  Hobs      AutoETS(auto=True)
                  Ovens     AutoETS(auto=True)
Food preservation Freezers  AutoETS(auto=True)
                  Fridges   AutoETS(auto=True)

In [9]:
# parallelization is enabled via the same config interface as for variables
# (same backend is used for both variables and instances or hierarchy levels)
forecaster = AutoETS(auto=True)

# backends are set via the backend:parallel config
forecaster.set_config(**{"backend:parallel": "loky"})  # or "multiprocessing", or "dask" (requires dask)
# backend params are set via the backend:parallel:params config
forecaster.set_config(**{"backend:parallel:params": {"n_jobs": 2}})  # passed to joblib.Parallel
# for documentation of the config interface, see set_config/get_config docstrings

AutoETS(auto=True)

In [10]:
# this is faster now!
forecaster.fit(y_train, fh=[1, 2, 3, 4])
y_pred = forecaster.predict()  # both fit and predict are parallelized

also works for:

* performance metrics (e.g., multivariate and hierarchical)
* transformation and preprocessing

side note: the same backend parameters are used for:

* embarrassingly parallel "special" estimators such as grid search, random search
* benchmarking and evaluation frameworks, e.g., `evaluate` for forecast benchmarks

estimator or function params are called:

* `backend`, string selecting backend, e.g., `loky`, `multiprocessing` or `dask`
* `backend_params`, dict with params passed to backend, e.g., `joblib.Parallel`

In [11]:
# example: parallelizing grid search
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.model_selection import ForecastingGridSearchCV
from sktime.performance_metrics.forecasting import MeanSquaredError
from sktime.split import ExpandingWindowSplitter

forecaster = ExponentialSmoothing()

cv = ExpandingWindowSplitter(fh=[1,2,3,4,5,6], initial_window=12, step_length=1)
param_grid = {
    "sp": [4, 6, 12],
    "seasonal": ["add", "mul"],
    "trend": ["add", "mul"],
    "damped_trend": [True, False],
}

gscv = ForecastingGridSearchCV(
    forecaster=forecaster,
    param_grid=param_grid,
    cv=cv,
    backend="loky",
    backend_params={"n_jobs": 2},
    verbose=1,
    scoring=MeanSquaredError(square_root=True),
)

## probabilistic forecasting, distribution outputs, skpro

recall probabilistic forecaster vignette:

In [12]:
from sktime.datasets import load_airline
from sktime.forecasting.theta import ThetaForecaster


# step 1: data specification
y = load_airline()
y_train = y.iloc[:-12]
y_test = y.iloc[-12:]
# step 2: specifying forecasting horizon
fh = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# step 3: specifying the forecasting algorithm
forecaster = ThetaForecaster(sp=12)
# step 4: fitting the forecaster
forecaster.fit(y_train, fh=fh)
# step 5: querying predictions
y_pred = forecaster.predict()

# for probabilistic forecasting:
#   call a probabilistic forecasting method after or instead of step 5
y_pred_int = forecaster.predict_interval(coverage=0.9)
y_pred_int

Number of airline passengers            
                                 0.9            
                               lower       upper
1960-01                   391.969490  433.006206
1960-02                   378.645958  428.083007
1960-03                   435.831197  492.435363
1960-04                   414.392525  477.353158
1960-05                   414.769824  483.501556
1960-06                   473.933061  547.987506
1960-07                   523.830411  602.849843
1960-08                   519.103324  602.793707
1960-09                   447.674785  535.788858
1960-10                   382.250311  474.576363
1960-11                   324.763079  421.117162
1960-12                   369.999658  470.220011

**probabilistic forecasting methods in `sktime`**:

* forecast intervals    - `predict_interval(fh=None, X=None, coverage=0.90)`
* forecast quantiles    - `predict_quantiles(fh=None, X=None, alpha=[0.05, 0.95])`
* forecast variance     - `predict_var(fh=None, X=None, cov=False)`
* distribution forecast - `predict_proba(fh=None, X=None, marginal=True)`

distribution forecasts have been reworked:

In [13]:
y_pred_distr = forecaster.predict_proba()
y_pred_distr

Normal(columns=Index(['Number of airline passengers'], dtype='object'),
       index=PeriodIndex(['1960-01', '1960-02', '1960-03', '1960-04', '1960-05', '1960-06',
             '1960-07', '1960-08', '1960-09', '1960-10', '1960-11', '1960-12'],
            dtype='period[M]'),
       mu=         Number of airline passengers
1960-01                    412.487848
1960-02                    403.364482
1960-03                    464.133280
1960-04                    445.872842
1960-05                    449.135690
1960-06                    510.960283
1960-07                    563.340127
1960-08                    560.948515
1960-09                    491.731822
1960-10                    428.413337
1960-11                    372.940120
1960-12                    420.109834,
       sigma=         Number of airline passengers
1960-01                     12.474276
1960-02                     15.027796
1960-03                     17.206445
1960-04                     19.138674
1960-05                     20.892963
1960-06                     22.510953
1960-07                     24.020202
1960-08                     25.440070
1960-09                     26.784777
1960-10                     28.065127
1960-11                     29.289562
1960-12                     30.464824)

scikit-base distribution object, first class citizen:

In [14]:
y_pred_distr.get_tags()

{'object_type': 'distribution',
 'python_version': None,
 'python_dependencies': None,
 'reserved_params': ['index', 'columns'],
 'capabilities:approx': ['pdfnorm'],
 'approx_mean_spl': 1000,
 'approx_var_spl': 1000,
 'approx_energy_spl': 1000,
 'approx_spl': 1000,
 'capabilities:exact': ['mean',
  'var',
  'energy',
  'pdf',
  'log_pdf',
  'cdf',
  'ppf'],
 'distr:measuretype': 'continuous'}

In [15]:
y_pred_distr.sample()

,Number of airline passengers
1960-01,421.790301
1960-02,398.448425
1960-03,439.921437
1960-04,480.989276
1960-05,454.925095
1960-06,516.498129
1960-07,568.554743
1960-08,585.966173
1960-09,508.967108
1960-10,454.107782


pandas-like interface:

In [16]:
y_pred_distr.index

PeriodIndex(['1960-01', '1960-02', '1960-03', '1960-04', '1960-05', '1960-06',
             '1960-07', '1960-08', '1960-09', '1960-10', '1960-11', '1960-12'],
            dtype='period[M]')

In [17]:
y_subset = y_pred_distr.iloc[[0, 1, 2]]
y_subset

Normal(columns=Index(['Number of airline passengers'], dtype='object'),
       index=PeriodIndex(['1960-01', '1960-02', '1960-03'], dtype='period[M]'),
       mu=array([[412.48784801],
       [403.36448248],
       [464.13327995]]),
       sigma=array([[12.4742762 ],
       [15.027796  ],
       [17.20644468]]))

distribution-defining functions:

In [18]:
import pandas as pd

x_df = pd.DataFrame([1, 1, 1], index=y_subset.index, columns=y_subset.columns)
y_subset.pdf(x_df)

,Number of airline passengers
1960-01,1.656111e-238
1960-02,5.684623e-158
1960-03,1.109667e-159


works seamlessly with probabilistic metrics:

In [19]:
from sktime.performance_metrics.forecasting.probabilistic import CRPS

crps = CRPS()

crps(y_test, y_pred_distr)

17.54994064789111

same interface also available for scikit-learn tabular regressors, with `skpro`!

see [sktime tutorial at pydata Amsterdam 2023](https://github.com/sktime/sktime-tutorial-pydata-Amsterdam-2023)

## modular time series distances, classifiers, aligners

Rich component relationships between object types!

* many classifiers, regressors, clusterers use distances or kernels
* distances and kernels are often composite, e.g., sum-of-distance, independent distance
* TS distances are often based on scalar multivariate distances (e.g., Euclidean)
* TS distances are often based on alignment, TS aligners are an estimator type!
* aligners internally typically use scalar uni/multivariate distances

example:

* 1-nn using `sklearn` nearest neighbors
* with multivariate dynamic time warping distance, from `dtw-python` library 
* on multivariate `"mahalanobis"` distance from `scipy`
* in `sktime` compatible interface, constructed from custom components

so, conceptually:

* we build an sequence alignment algorithm (`dtw-python`) using `scipy` Mahalanobis dist
* we get the distance matrix computation from alignment algorithm
* we use that distance matrix in `sklearn` knn
* together this is a time series classifier!

In [20]:
from sktime.alignment.dtw_python import AlignerDTWfromDist
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels.compose_from_align import DistFromAligner
from sktime.dists_kernels.scipy_dist import ScipyDist

# Mahalanobis distance on R^n
mahalanobis_dist = ScipyDist(metric="mahalanobis")  # uses scipy distances

# pairwise multivariate aligner from dtw-python with Mahalanobis distance
mw_aligner = AlignerDTWfromDist(mahalanobis_dist)  # uses dtw-python

# turning this into alignment distance on time series
dtw_dist = DistFromAligner(mw_aligner)  # interface mutation to distance

# and using this distance in a k-nn classifier
clf = KNeighborsTimeSeriesClassifier(distance=dtw_dist)  # uses sklearn knn

works seamlessly with `get_params`, `set_params` for tuning!

In [21]:
clf.get_params()

{'algorithm': 'brute',
 'distance': DistFromAligner(aligner=AlignerDTWfromDist(dist_trafo=ScipyDist(metric='mahalanobis'))),
 'distance_mtype': None,
 'distance_params': None,
 'leaf_size': 30,
 'n_jobs': None,
 'n_neighbors': 1,
 'pass_train_distances': False,
 'weights': 'uniform',
 'distance__aligner': AlignerDTWfromDist(dist_trafo=ScipyDist(metric='mahalanobis')),
 'distance__aligner__dist_trafo': ScipyDist(metric='mahalanobis'),
 'distance__aligner__open_begin': False,
 'distance__aligner__open_end': False,
 'distance__aligner__step_pattern': 'symmetric2',
 'distance__aligner__window_type': 'none',
 'distance__aligner__dist_trafo__colalign': 'intersect',
 'distance__aligner__dist_trafo__metric': 'mahalanobis',
 'distance__aligner__dist_trafo__metric_kwargs': None,
 'distance__aligner__dist_trafo__p': 2,
 'distance__aligner__dist_trafo__var_weights': None}

all object types are first class citizens in sktime!

* `"transformer-panel"` - time series distances, kernels, pairwise transformers on panel data
* `"transformer-pairwise"` for all pairwise transformers on tabular data, e.g., scalar distance
* `"aligner"` for all time series aligners
* `"transformer"` for all transformers, these can be composed with all the above

In [22]:
from sktime.registry import all_estimators

all_estimators("transformer-pairwise-panel", as_dataframe=True, return_tags=["pwtrafo_type"])

,name,object,pwtrafo_type
0,AggrDist,<class 'sktime.dists_kernels.compose_tab_to_pa...,distance
1,CombinedDistance,<class 'sktime.dists_kernels.algebra.CombinedD...,distance
2,ConstantPwTrafoPanel,<class 'sktime.dists_kernels.dummy.ConstantPwT...,distance
3,CtwDistTslearn,<class 'sktime.dists_kernels.ctw.CtwDistTslearn'>,distance
4,DistFromAligner,<class 'sktime.dists_kernels.compose_from_alig...,distance
5,DistFromKernel,<class 'sktime.dists_kernels.dist_to_kern.Dist...,distance
6,DtwDist,<class 'sktime.dists_kernels.dtw._dtw_sktime.D...,distance
7,DtwDistTslearn,<class 'sktime.dists_kernels.dtw._dtw_tslearn....,distance
8,DtwPythonDist,<class 'sktime.dists_kernels.dtw._dtw_python.D...,distance
9,EditDist,<class 'sktime.dists_kernels.edit_dist.EditDist'>,distance


In [23]:
from sktime.registry import all_estimators

all_estimators("aligner", as_dataframe=True)

,name,object
0,AlignerDTW,<class 'sktime.alignment.dtw_python.AlignerDTW'>
1,AlignerDTWfromDist,<class 'sktime.alignment.dtw_python.AlignerDTW...
2,AlignerDtwNumba,<class 'sktime.alignment.dtw_numba.AlignerDtwN...
3,AlignerEditNumba,<class 'sktime.alignment.edit_numba.AlignerEdi...
4,AlignerLuckyDtw,<class 'sktime.alignment.lucky.AlignerLuckyDtw'>
5,AlignerNaive,<class 'sktime.alignment.naive.AlignerNaive'>


see [sktime tutorial at pydata London 2023](https://github.com/sktime/sktime-tutorial-pydata-london-2023)

## Benchmarking - comparing estimator performance

the `benchmarking` module allows you to set up experiments to:

* compare the performance of one or more algorithms
* over one or multiple datasets
* against one or multiple performance metrics
* for a benchmark configuration defined by temporal resampling scheme


`sktime`'s `benchmarking` module is designed to:

* provide a high-level specification language
* prevent mistakes by abstracting away "dangerous" implementation details
* allow reproducible sharing of experiment setups and results

Any `sktime` compatible object can be plugged in!

Use `sktime` extension templates to add custom objects to experiment!

(this cell requires `kotsu` in the environment)

In [ ]:
from sktime.benchmarking.forecasting import ForecastingBenchmark
from sktime.datasets import load_airline
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import MeanSquaredPercentageError

# set up benchmark
benchmark = ForecastingBenchmark()

# add competing estimators
benchmark.add_estimator(
    estimator=NaiveForecaster(strategy="mean", sp=12),
    estimator_id="NaiveForecaster-mean-v1",
)
benchmark.add_estimator(
    estimator=NaiveForecaster(strategy="last", sp=12),
    estimator_id="NaiveForecaster-last-v1",
)

# define tasks, for forecasting:
# backtesting schema, cv splitter, scorer, data
cv_splitter = ExpandingWindowSplitter(
    initial_window=24,
    step_length=12,
    fh=12,
)
scorers = [MeanSquaredPercentageError()]
dataset_loaders = [load_airline]

# add task
for dataset_loader in dataset_loaders:
    benchmark.add_task(
        dataset_loader,
        cv_splitter,
        scorers,
    )

# run the experiment, write to csv
results_df = benchmark.run("./forecasting_results.csv")
results_df.T

for forecasting, use `evaluate` utility for smaller runs

see [sktime tutorial at pycon Prague 2023](https://github.com/sktime/sktime-tutorial-pydata-global-2023)

---

# Marketplace and deployment features

* estimator search, estimator tags
* scikit-base interface
* blueprint serialization and sharing
* fitted estimator serialization and sharing
* mlflow deployment via custom flavour

## listing estimators, estimator search, estimator tags

* all objects now "first class citizens" with a type - scikit-base objects
* use `all_estimators` for search subset

example: list all forecasters (`sktime` native scope)

In [25]:
from sktime.registry import all_estimators

all_estimators("forecaster", as_dataframe=True)

,name,object
0,ARCH,<class 'sktime.forecasting.arch._uarch.ARCH'>
1,ARDL,<class 'sktime.forecasting.ardl.ARDL'>
2,ARIMA,<class 'sktime.forecasting.arima.ARIMA'>
3,AutoARIMA,<class 'sktime.forecasting.arima.AutoARIMA'>
4,AutoETS,<class 'sktime.forecasting.ets.AutoETS'>
...,...,...
65,UpdateRefitsEvery,<class 'sktime.forecasting.stream._update.Upda...
66,VAR,<class 'sktime.forecasting.var.VAR'>
67,VARMAX,<class 'sktime.forecasting.varmax.VARMAX'>
68,VECM,<class 'sktime.forecasting.vecm.VECM'>


or, list all splitters:

In [26]:
all_estimators("splitter", as_dataframe=True)

,name,object
0,CutoffSplitter,<class 'sktime.split.cutoff.CutoffSplitter'>
1,ExpandingGreedySplitter,<class 'sktime.split.expandinggreedy.Expanding...
2,ExpandingWindowSplitter,<class 'sktime.split.expandingwindow.Expanding...
3,SameLocSplitter,<class 'sktime.split.sameloc.SameLocSplitter'>
4,SingleWindowSplitter,<class 'sktime.split.singlewindow.SingleWindow...
5,SlidingWindowSplitter,<class 'sktime.split.slidingwindow.SlidingWind...
6,TemporalTrainTestSplitter,<class 'sktime.split.temporal_train_test_split...
7,TestPlusTrainSplitter,<class 'sktime.split.testplustrain.TestPlusTra...


all classes, objects come with tags:

In [27]:
# class tags
from sktime.forecasting.arima import ARIMA

ARIMA.get_class_tags()
# interesting for users:
# object_type tells us this is a forecaster
# capability tags, e.g., "capability:insample", "capability:pred_int"

{'python_dependencies_alias': {'scikit-learn': 'sklearn'},
 'object_type': 'forecaster',
 'scitype:y': 'univariate',
 'ignores-exogeneous-X': False,
 'capability:insample': True,
 'capability:pred_int': True,
 'capability:pred_int:insample': True,
 'handles-missing-data': True,
 'y_inner_mtype': 'pd.Series',
 'X_inner_mtype': 'pd.DataFrame',
 'requires-fh-in-fit': False,
 'X-y-must-have-same-index': True,
 'enforce_index_type': None,
 'fit_is_empty': False,
 'python_version': None,
 'python_dependencies': 'pmdarima'}

In [28]:
# object tags
from sktime.forecasting.naive import NaiveForecaster

NaiveForecaster().get_tags()
# same tags
# values may depend on the object parameters, e.g., "handles-missing-data"
# class tags are "most general" capabilities

{'python_dependencies_alias': {'scikit-learn': 'sklearn'},
 'object_type': 'forecaster',
 'scitype:y': 'univariate',
 'ignores-exogeneous-X': True,
 'capability:insample': True,
 'capability:pred_int': True,
 'capability:pred_int:insample': True,
 'handles-missing-data': True,
 'y_inner_mtype': 'pd.Series',
 'X_inner_mtype': 'pd.DataFrame',
 'requires-fh-in-fit': False,
 'X-y-must-have-same-index': True,
 'enforce_index_type': None,
 'fit_is_empty': False,
 'python_version': None,
 'python_dependencies': None,
 'capability:pred_var': True}

produce table with class tags:

In [29]:
from sktime.registry import all_estimators

# list all forecasters, in a table, with two added columns
# capability:insample - can produce in-sample forecasts?
# capability:pred_int - can produce prediction intervals?
all_estimators(
    "forecaster", as_dataframe=True, return_tags=["capability:insample", "capability:pred_int"]
)

,name,object,capability:insample,capability:pred_int
0,ARCH,<class 'sktime.forecasting.arch._uarch.ARCH'>,True,True
1,ARDL,<class 'sktime.forecasting.ardl.ARDL'>,True,False
2,ARIMA,<class 'sktime.forecasting.arima.ARIMA'>,True,True
3,AutoARIMA,<class 'sktime.forecasting.arima.AutoARIMA'>,True,True
4,AutoETS,<class 'sktime.forecasting.ets.AutoETS'>,True,True
...,...,...,...,...
65,UpdateRefitsEvery,<class 'sktime.forecasting.stream._update.Upda...,True,False
66,VAR,<class 'sktime.forecasting.var.VAR'>,True,True
67,VARMAX,<class 'sktime.forecasting.varmax.VARMAX'>,True,False
68,VECM,<class 'sktime.forecasting.vecm.VECM'>,True,True


filter for class tags:

In [30]:
# list all forecasters that can produce probabilistic forecasts
all_estimators(
    "forecaster", as_dataframe=True, filter_tags={"capability:pred_int": True}
)
# of course you can do this with simple pandas filtering too,
# or anything else you want to do with pandas, but it avoids tedious wrangling

,name,object
0,ARCH,<class 'sktime.forecasting.arch._uarch.ARCH'>
1,ARIMA,<class 'sktime.forecasting.arima.ARIMA'>
2,AutoARIMA,<class 'sktime.forecasting.arima.AutoARIMA'>
3,AutoETS,<class 'sktime.forecasting.ets.AutoETS'>
4,BATS,<class 'sktime.forecasting.bats.BATS'>
5,BaggingForecaster,<class 'sktime.forecasting.compose._bagging.Ba...
6,ColumnEnsembleForecaster,<class 'sktime.forecasting.compose._column_ens...
7,ConformalIntervals,<class 'sktime.forecasting.conformal.Conformal...
8,DynamicFactor,<class 'sktime.forecasting.dynamic_factor.Dyna...
9,FhPlexForecaster,<class 'sktime.forecasting.compose._fhplex.FhP...


roadmap, contribute!

* easy way to specify variable scope across packages, 1st, 2nd, and 3rd party
* updating estimator overview frontend

## sharing model blueprints and fitted models

how to share these with your friends?

* model blueprint specs, e.g., equivalent of spec `Pipeline([("foo", Foo()), ("bar", Bar(42))])`
* fitted models, e.g., state of `my_pipe.fit(y)` after the `fit` - specific to data!

### sharing model blueprints

blueprint specs can be serialized using simple string print - this contains all information!

In [31]:
# let's define an example pipeline
from sktime.forecasting.compose._pipeline import TransformedTargetForecaster
from sktime.forecasting.naive import NaiveForecaster
from sktime.transformations.series.impute import Imputer

pipe = TransformedTargetForecaster(
    steps=[
        ("imputer", Imputer()),
        ("forecaster", NaiveForecaster()),
    ]
)

In [32]:
# serialize the pipeline to a string
# this is useful for logging and sharing
# pipe_str can be saved to a file, database, or shared over the internet
pipe_str = str(pipe)
pipe_str

"TransformedTargetForecaster(steps=[('imputer', Imputer()),\n                                   ('forecaster', NaiveForecaster())])"

for pseudo-random determinism, set any `random_state` parameters in the estimators

to deserialize, use `registry.craft` in the same python environment

for python environment, e.g., use `pip freeze`

In [33]:
from sktime.registry import craft

pipe_new = craft(pipe_str)
pipe_new

TransformedTargetForecaster(steps=[('imputer', Imputer()),
                                   ('forecaster', NaiveForecaster())])

this is the same estimator blueprint as `pipe`!

To compare blueprint, simply use the `==` operator (this is a `scikit-base` feature)

In [34]:
pipe_new == pipe

True

sharing process:

* origin shares `pipe_str = str(pipe)` or `str(my_estimator)` and `pip freeze > requirements.txt` output
* recipient installs env from `requirements.txt` and runs `craft(pipe_str)` in that env

For custom estimators, in addition, the custom module needs to be shared.

Highly complex estimators can consist of multiple definition blocks - this is also supported by `craft` as follows.

Instead of a string conversion, we can also serialize:

In [35]:
# pipe_spec is a string representation of the pipeline
# it can be stored in a file or a database like this
# the "return" statement indicates which object we store
# temporary variables like pipe, cv can be defined
pipe_spec = """
pipe = TransformedTargetForecaster(steps=[
    ("imputer", Imputer()),
    ("forecaster", NaiveForecaster())])
cv = ExpandingWindowSplitter(
    initial_window=24,
    step_length=12,
    fh=[1, 2, 3])

return ForecastingGridSearchCV(
    forecaster=pipe,
    param_grid=[{
        "forecaster": [NaiveForecaster(sp=12)],
        "forecaster__strategy": ["drift", "last", "mean"],
    },
    {
        "imputer__method": ["mean", "drift"],
        "forecaster": [ThetaForecaster(sp=12)],
    },
    {
        "imputer__method": ["mean", "median"],
        "forecaster": [ExponentialSmoothing(sp=12)],
        "forecaster__trend": ["add", "mul"],
    },
    ],
    cv=cv,
    n_jobs=-1)
"""

In [36]:
craft(pipe_spec)

ForecastingGridSearchCV(cv=ExpandingWindowSplitter(fh=[1, 2, 3],
                                                   initial_window=24,
                                                   step_length=12),
                        forecaster=TransformedTargetForecaster(steps=[('imputer',
                                                                       Imputer()),
                                                                      ('forecaster',
                                                                       NaiveForecaster())]),
                        n_jobs=-1,
                        param_grid=[{'forecaster': [NaiveForecaster(sp=12)],
                                     'forecaster__strategy': ['drift', 'last',
                                                              'mean']},
                                    {'forecaster': [ThetaForecaster(sp=12)],
                                     'imputer__method': ['mean', 'drift']},
                                    {'forecaster': [ExponentialSmoothing(sp=12)],
                                     'forecaster__trend': ['add', 'mul'],
                                     'imputer__method': ['mean', 'median']}])

some estimators require soft dependencies to be installed at `craft`

query required dependencies can *before* construction via `deps`:

In [37]:
from sktime.registry import deps

deps(pipe_spec)

['statsmodels']

(if `pip freeze` is not ehough)

`imports` can be used to print a full import block:

In [38]:
from sktime.registry import imports

imports(pipe_spec)  # the result can be copied above the spec in to a jupyter cell

'from sktime.forecasting.compose._pipeline import TransformedTargetForecaster\nfrom sktime.forecasting.exp_smoothing import ExponentialSmoothing\nfrom sktime.forecasting.model_selection._tune import ForecastingGridSearchCV\nfrom sktime.forecasting.naive import NaiveForecaster\nfrom sktime.forecasting.naive import NaiveForecaster\nfrom sktime.forecasting.theta import ThetaForecaster\nfrom sktime.split.expandingwindow import ExpandingWindowSplitter\nfrom sktime.transformations.series.impute import Imputer'

### Persisting fitted models

to persist a fitted model:

In [39]:
from sktime.datasets import load_airline

y = load_airline()

In [40]:
# example pipeline
from sktime.forecasting.compose._pipeline import TransformedTargetForecaster
from sktime.forecasting.naive import NaiveForecaster
from sktime.transformations.series.impute import Imputer

pipe = TransformedTargetForecaster(
    steps=[
        ("imputer", Imputer()),
        ("forecaster", NaiveForecaster()),
    ]
)

pipe.fit(y, fh=[1, 2, 3])

TransformedTargetForecaster(steps=[('imputer', Imputer()),
                                   ('forecaster', NaiveForecaster())])

to serialize fitted objects, use `save` - default is `pkl`, but may differ for deep learning

* no args produces in-memory object
* `str` or `Path` arg will serialize to file

In [41]:
pipe_mem = pipe.save()
# pipe_mem is a pickle

to deserialize use the `load` method on the memory object or a `str`, `Path`:

In [42]:
from sktime.base import load

pipe_new = load(pipe_mem)

the loaded object can be used for prediction now.

In [43]:
pipe_new.predict()

1961-01    432.0
1961-02    432.0
1961-03    432.0
Freq: M, Name: Number of airline passengers, dtype: float64

for more, see [sktime tutorial at pycon Prague 2023](https://github.com/sktime/sktime-tutorial-pydata-global-2023)

### mlflow custom flavour

with `mlflow` / `mlflavors`:

* use `mlflow` context manager `start_run`
* results are logged/saved using standard `mlflow.log_params`, `log_metrics`
* model is logged/saved using `mlflavors.sktime.log_model`

for further use (load), get artefact URI using `get_artifact_uri`

Example: save fitted model, model parameters, and results of this experiment to server

* fit `NaiveForecaster` on longley data (with exogenous vars)
* evaluate via MAE and MAPE

In [ ]:
import json

import mlflavors
import mlflow
from sktime.datasets import load_longley
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import (
    mean_absolute_error,
    mean_absolute_percentage_error,
)


ARTIFACT_PATH = "model"

with mlflow.start_run() as run:
    y, X = load_longley()
    y_train, y_test, X_train, X_test = temporal_train_test_split(y, X)

    forecaster = NaiveForecaster()
    forecaster.fit(
        y_train,
        X=X_train,
        fh=[1, 2, 3, 4],
    )

    # Extract parameters
    parameters = forecaster.get_params()

    # Evaluate model
    y_pred = forecaster.predict(X=X_test)
    metrics = {
        "mae": mean_absolute_error(y_test, y_pred),
        "mape": mean_absolute_percentage_error(y_test, y_pred),
    }

    print(f"Parameters: \n{json.dumps(parameters, indent=2)}")
    print(f"\nMetrics: \n{json.dumps(metrics, indent=2)}")

    # Log parameters and metrics
    mlflow.log_params(parameters)
    mlflow.log_metrics(metrics)

    # Log model to MLflow tracking server
    mlflavors.sktime.log_model(
        sktime_model=forecaster,
        artifact_path=ARTIFACT_PATH,
    )
    
    # Return model uri from the current run
    model_uri = mlflow.get_artifact_uri(ARTIFACT_PATH)
    
# Print the run id wich is used below for serving the model to a local REST API endpoint
print(f"\nMLflow run id:\n{run.info.run_id}")

loading via `load_model` (below) or `pyfunc`:

In [ ]:
loaded_model = mlflavors.sktime.load_model(model_uri=model_uri)
print(loaded_model.predict_interval(fh=[1, 2, 3], X=X_test, coverage=[0.9, 0.95]))

see [sktime tutorial at ODSC Europe 2023](https://github.com/sktime/sktime-tutorial-ODSC-Europe-2023)

---

### 2022-2023 highlights seen today

#### Forecasting

* streamlined interface
* pipelines introduction
* new: graphical pipeline

#### Advanced modelling

* extended parallelism, including parallel broadcasting to hierarchical data
* fully distributional probabilistic forecasts and metrics, skpro
* composable time series classifiers, regressors, distances, time series aligners
* benchmarking frameworks for comparing estimator performance

#### Marketplace and deployment features

* estimator search, estimator tags
* scikit-base interface for multiple libraries
* blueprint serialization and sharing
* fitted estimator serialization and sharing
* mlflow deployment via custom flavour

---
### Credits: notebook 4 - new feature vignettes

notebook creation: fkiraly

some vignettes based on previous workshops, as linked

General credit also to `sklearn` and `sktime` contributors